In [2]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import pickle
import os


In [3]:
typ = 'mb_' # _sme #for SME
df = pd.read_excel(typ+'ipo-reviews-final_data.xlsx')
model_name = 'markussagen/xlm-roberta-longformer-base-4096'#"microsoft/deberta-v3-base"

In [4]:
os.environ['WANDB_DISABLED'] = 'true'

In [5]:
!pip install -q transformers pandas openpyxl datasets evaluate bitsandbytes

In [6]:
!pip install -q -U bitsandbytes

In [7]:
# !git clone https://github.com/NVIDIA/apex.git
# !pip install -q -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" apex

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
tokenizer = AutoTokenizer.from_pretrained(model_name)
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=4096)

tokenizer_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/773 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [10]:
recom_di = {'Avoid' : 0, 'Neutral' :1, 'May apply' : 2, 'Apply' : 3}

In [ ]:
df['target'] = df['Recommendation'].apply(lambda x : recom_di[x])
y = 'target'
feature = 'input_text'
df[feature] = df['summary_of_all_answers'].to_list()
train_df = df[df['year']<2024][['input_text', y]]
test_df = df[df['year']>=2024][['input_text', y]]
full_df = df[['input_text', y]]

train_df['input_text'].fillna('', inplace=True)
full_df['input_text'].fillna('', inplace=True)



In [12]:
import os
os.environ["WANDB_DISABLED"] = "true"

import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import os
import shutil



train_data = pd.DataFrame({'text': train_df[feature].to_list(), 'labels': train_df[y].to_list()})
train_dataset = Dataset.from_pandas(train_data)
train_dataset = train_dataset.map(preprocess_function, batched=True)



full_data = pd.DataFrame({'text': full_df[feature].to_list(), 'labels': full_df[y].to_list()})
full_dataset = Dataset.from_pandas(full_data)
full_dataset = full_dataset.map(preprocess_function, batched=True)

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(recom_di))#, load_in_4bit = True)

training_args = TrainingArguments(
output_dir=f"./results-{feature}",          # Output directory (can be ignored, we don't save models)
evaluation_strategy="no",                   # No evaluation during training
learning_rate=2e-5,                         # Learning rate
per_device_train_batch_size=1,              # Batch size for training
per_device_eval_batch_size=1,               # Batch size for evaluation
num_train_epochs=5,                         # Number of epochs
gradient_accumulation_steps=4,
weight_decay=0.01,                          # Strength of weight decay
logging_dir=None,                           # No logging
save_total_limit=0,                         # No saving of checkpoints
save_steps=0,                               # No saving steps
load_best_model_at_end=False                # No need to load best model
)

trainer = Trainer(
model=model,
args=training_args,
train_dataset=train_dataset,
tokenizer=tokenizer
)


trainer.train()

predictions = trainer.predict(full_dataset)
preds_proba = torch.softmax(torch.tensor(predictions.predictions), dim=-1).numpy()



preds_labels = preds_proba.argmax(axis=1)
full_df['predicted_labels'] = preds_labels

        


Map:   0%|          | 0/1397 [00:00<?, ? examples/s]

Map:   0%|          | 0/1830 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at markussagen/xlm-roberta-longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipykernel_23/2644503875.py:52: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  traine

/tmp/ipykernel_23/2644503875.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df['predicted_labels'] = preds_labels


In [ ]:
full_df.to_csv('/working/' + typ + model_name.replace('/', '__')+ 'predictions_full.csv', index = False)

model.save_pretrained('/working/' + typ + model_name.replace('/', '__')+ 'model-fine-tuned')
tokenizer.save_pretrained('/working/' + typ + model_name.replace('/', '__') + 'tokenizer-fine-tuned')